In [ ]:
import os
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
from pysph.solver.utils import iter_output, load
from scipy.signal import argrelextrema
pi, sin, cos = np.pi, np.sin, np.cos

OUTPUT_DIR = ''

In [ ]:
def ramp(t_star):
    if t_star>=0.0 and t_star<0.1:
        return t_star*10.
    elif t_star>=0.1 and t_star<0.9:
        return 1.
    elif t_star>=0.9 and t_star<=1.:
        return (1.-t_star)*10.
    else:
        return 0.
    
def get_sub_fldrs(path):
    folders = []
    for folder in os.listdir(path):
        if os.path.isdir(
            os.path.join(path, folder)
        ):
            # Get absolute path of folder
            abs_path = os.path.abspath(os.path.join(path, folder))
            folders.append(abs_path)
    folders = [fldr for fldr in folders if os.path.basename(fldr).startswith('c0_')]
    return folders

def get_opts_from_fldr(fname):
    fname = os.path.basename(fname)
    opts = {}
    expected_keys = ['c0', 'dtmul', 'nx', 're']
    kwrds = fname.split('_')
    kwrds.remove('tsph')
    # Get the values of the expected keys
    for key in expected_keys:
        try:
            idx = kwrds.index(key)
            opts[key] = kwrds[idx+1]
            kwrds.remove(key)
            kwrds.remove(opts[key])
        except ValueError:
            pass
    opts['integrator'] = kwrds[0]
    return opts

def get_master_dict(main_dict):
    """
    Given a list of dictionaries, return a dictionary of the form:
        {'key1': [val1, val2, ...], 'key2': [val1, val2, ...], ...}
    """
    master_dict = {}
    for d in main_dict:
        for key, val in d.items():
            if key not in master_dict:
                master_dict[key] = [val]
            else:
                master_dict[key].append(val)
    # Remove duplicate values
    for key, val in master_dict.items():
        master_dict[key] = list(set(val))
        
    return master_dict
    
def get_results_file(path):
    for file in os.listdir(path):
        if file.endswith('results.npz'):
            return os.path.join(path, file)

def select_subfolder(subfolders, **kwargs):
    """
    Given a list of subfolders, and a dictionary of the form:
        {'key1': val1, 'key2': val2, ...}
    return a list of subfolders that match the criteria.
    Example:
        select_subfolder(folders, re='100')
    """
    if len(kwargs) == 0:
        return subfolders

    # Make a copy of subfolders
    selected = subfolders.copy()
    # Loop over all keys in kwargs
    for key, val in kwargs.items():
        if len(selected) == 0:
            return []
        # Remove folders that don't match the criteria
        for fldr in subfolders:
            # tf = os.path.basename(fldr)
            opts = get_opts_from_fldr(fldr)
            # print(tf)
            # print(opts)
            if key not in opts.keys() and (fldr in selected):
                selected.remove(fldr)
            else:
                if opts[key] != val and (fldr in selected):
                    selected.remove(fldr)
    return selected

In [35]:
def plotter(
    parent_dir, plot_type, ylim=None, exact=False,
    plt_ramp=False, ramp_a=1.3, save_fig=False, 
    plot_method='plot', **kwargs
):
    plot_types = ['decay', 'l1', 'linf', 'p_l1', 'ke']
    if plot_type not in plot_types:
        raise ValueError(
            f'Invalid plot type: {plot_type}. Valid types are: {plot_types}'
        )
    subfolders = get_sub_fldrs(parent_dir)
    subfolders = select_subfolder(subfolders, **kwargs)

    if plot_method == 'semilogy':
        plotter_func = plt.semilogy
    elif plot_method == 'plot':
        plotter_func = plt.plot
    else:
        raise ValueError(
            f'Invalid plot method: {plot_method}. Valid methods are: plot, semilogy'
        )

    # Get maximum time
    t_sim_max = []
    for folder in subfolders:
        results_file = get_results_file(folder)
        if results_file:
            data = np.load(results_file)
            t_sim_max.append(data['t'][-1])
    t_sim_max_id = np.argmax(t_sim_max)
    
    plot_exact = exact
    plt.figure(figsize=(8, 6))
    for folder in subfolders:
        results_file = get_results_file(folder)
        if results_file:
            data = np.load(results_file)
            t = data['t']
            plt_label = os.path.basename(folder)
            
            if plot_type == 'decay':
                if plot_exact:
                    _data = get_results_file(subfolders[t_sim_max_id])
                    _data = np.load(_data)
                    _t, decay_ex = _data['t'], _data['decay_ex']
                    plotter_func(_t, decay_ex, 'k--', label='Exact')
                    plot_exact = False
                plotter_func(t, data['decay'], label=plt_label)
            elif plot_type == 'l1':
                plotter_func(t, data['l1'], label=plt_label)
            elif plot_type == 'linf':
                plotter_func(t, data['linf'], label=plt_label)
            elif plot_type == 'p_l1':
                plotter_func(t, data['p_l1'], label=plt_label)
            elif plot_type == 'ke':
                if plot_exact:
                    _data = get_results_file(subfolders[t_sim_max_id])
                    _data = np.load(_data)
                    _t, ke_ex = _data['t'], _data['ke_ex']
                    plotter_func(_t, ke_ex, 'k--', label='Exact')
                    plot_exact = False
                plotter_func(t, data['ke'], label=plt_label)
        else:
            print(f'No results file found in {folder}')
    
    plot_exact = exact
    plot_ramp = plt_ramp
    for folder in subfolders:
        results_file = get_results_file(folder)
        if results_file:
            data = np.load(results_file)
            t = data['t']

            if plot_ramp:
                _t = np.linspace(0, t[-1], 700)
                f = [ramp(i) for i in _t]
                f = np.array(f)*ramp_a
                label = 'External force'
                if ramp_a == 1.3:
                    label += ' (Exact)'
                else:
                    label += ' (Trendline)'
                plotter_func(_t, f, 'k--', label=label)
                plot_ramp = False

            if plot_type == 'ke':
                if plot_exact:
                    maxima = argrelextrema(data['ke_ex'], np.greater)
                    if len(maxima[0]) > 0:
                        plotter_func(t[maxima], data['ke_ex'][maxima], 'o')
                        plot_exact = False

                # Plot local maximas as dots
                maxima = argrelextrema(data['ke'], np.greater)
                if len(maxima[0]) > 0:
                    plotter_func(t[maxima], data['ke'][maxima], 'o')

    
    if plot_type == 'ke':
        plt.ylim(0, 0.4)

    if ylim:
        plt.ylim(ylim)

    # Title is the keys=values in kwargs
    title = plot_type
    for key, val in kwargs.items():
        title += f" ({key}={val}) "

    plt.grid()
    plt.title(title)
    plt.xlabel('t')
    plt.legend()
    labels = [l.get_label() for l in plt.gca().get_lines()]
    if save_fig:
        fname = os.path.join(OUTPUT_DIR, f'{title}.png')
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        print(f'Figure saved to {fname}')


In [52]:
def get_output_files(dir):
    files = os.listdir(dir)
    # Select only the files that end with .hdf5
    files = [os.path.join(dir, f) for f in files if f.endswith('.hdf5')]
    def _get_file_counter(f):
        return int(os.path.basename(f).split('_')[-1].split('.')[0])
    files = sorted(files, key=_get_file_counter)
    return files

def guess_subplot_size(N):
    """
    Given N, suggest a subplot size that is close to a square or rectangle
    """
    rows = int(np.sqrt(N))
    cols = int(np.ceil(N / rows))
    return rows, cols

def plot_vel_field(parent_dir, last_common_tf=False, save_fig=False, **kwargs):
    subfolders = get_sub_fldrs(parent_dir)
    subfolders = select_subfolder(subfolders, **kwargs)
    N = len(subfolders)

    # Get the last common tf
    t_err = [4]*N
    if not last_common_tf:
        idx = -1
    else:
        _n = []
        for i, folder in enumerate(subfolders):
            output_files = get_output_files(folder)
            _n.append(len(output_files))
            if len(output_files) != 50:
                data = load(output_files[-1])
                t = data['solver_data']['t']
                t_err[i] = t
        idx = min(_n) - 1
    
    # Create a figure with N subplots
    rows, cols = guess_subplot_size(N)
    fig, axes = plt.subplots(rows, cols, figsize=(cols*6, rows*4))
    cmap=cm.get_cmap('viridis')
    normalizer=Normalize(0,1)
    im=cm.ScalarMappable(norm=normalizer)
    if N > 1:
        axes = axes.flatten()
    else:
        axes = np.array([axes])
    for i, folder in enumerate(subfolders):
        output_files = get_output_files(folder)
        data = load(output_files[idx])
        t = data['solver_data']['t']
        array = data['arrays']['fluid']
        x, y = array.get('x', 'y')
        u, v = array.get('u', 'v')
        vmag = np.sqrt(u**2 + v**2)
        axes[i].scatter(x, y, c=vmag, s=1, cmap=cmap, norm=normalizer)
        # Title is the keys=values in kwargs
        opts = get_opts_from_fldr(folder)
        intg = opts['integrator']
        dtmf = opts['dtmul']

        msg = ''
        if t_err[i] < 4:
            msg = f' (error at t={t_err[i]:.2f})'
        title = fr"t={t:.2f} ({intg} - {dtmf}$\Delta t$) {msg}"
        axes[i].set_title(title)
    
    # Set main title with kwargs
    fig.subplots_adjust(right=0.8)
    fig.colorbar(im, ax=axes.ravel().tolist())
    title = 'Velocity field'
    for key, val in kwargs.items():
        title += f" ({key}={val}) "
    if last_common_tf:
        min_t_err = min(t_err)
        title += fr' (Last common $t_f$={min_t_err:.2f})'
    # Title should be bold and centered
    fig.suptitle(
        title, fontsize=16, fontweight='bold', horizontalalignment='right',
    )

    if save_fig:
        title = title.split('(Last')[0]
        title = title.strip()
        if last_common_tf:
            title += ' (Last common t_f)'
        fname = os.path.join(OUTPUT_DIR, f'{title}.png')
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
        print(f'Figure saved to {fname}')
# plot_vel_field(fdir, last_common_tf=True, save_fig=0, re=str(10_00))

In [47]:
def set_output_dir(dir):
    fdir = os.path.basename(os.path.abspath(dir))
    global OUTPUT_DIR
    OUTPUT_DIR = os.path.join(os.getcwd(), 'manuscript', 'code_figures', fdir)
    print(f'Output directory: {OUTPUT_DIR}')
    if not os.path.exists(OUTPUT_DIR):
        os.mkdir(OUTPUT_DIR)

In [30]:
fdir = 'outputs/tgv_2d_scheme_comparison'
set_output_dir(fdir)

fldrs = get_sub_fldrs(fdir)
temp = [get_opts_from_fldr(fldr) for fldr in fldrs]
master_dict = get_master_dict(temp)
pprint(master_dict)

CH = ['decay', 'l1', 'linf', 'p_l1', 'ke']
for ch in CH:
    for re in master_dict['re']:
        plotter(fdir, ch, exact=True, save_fig=True, re=str(re))

for re in master_dict['re']:
    plot_vel_field(fdir, last_common_tf=True, save_fig=True, re=str(re))
    plot_vel_field(fdir, last_common_tf=False, save_fig=True, re=str(re))

Output directory: d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison
{'c0': ['40'],
 'dtmul': ['3', '4', '2', '6', '8', '1'],
 'integrator': ['rk3', 'rk2', 'rk4', 'pec'],
 'nx': ['200'],
 're': ['10000', '50000', '1000']}
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\decay (re=10000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\decay (re=50000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\decay (re=1000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\l1 (re=10000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\l1 (re=50000) .png
Figure saved to 

C:\Users\prajw\AppData\Local\Temp\ipykernel_22688\452576616.py:90: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.ylim(0, 0.4)


Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\ke (re=10000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\ke (re=50000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\ke (re=1000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\Velocity field (re=10000) (Last common t_f).png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\Velocity field (re=10000).png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_2d_scheme_comparison\Velocity field (re=50000) (Last common t_f).png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figu

In [55]:
fdir = 'outputs/tgv_ext_force_scheme_comparison'
set_output_dir(fdir)

fldrs = get_sub_fldrs(fdir)
temp = [get_opts_from_fldr(fldr) for fldr in fldrs]
master_dict = get_master_dict(temp)
pprint(master_dict)

CH = ['ke', 'decay']
for ch in CH:
    for re in master_dict['re']:
        plotter(fdir, ch, exact=False, save_fig=True, re=str(re))

for re in master_dict['re']:
    plot_vel_field(fdir, last_common_tf=True, save_fig=True, re=str(re))

Output directory: d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_ext_force_scheme_comparison
{'c0': ['40'],
 'dtmul': ['4'],
 'integrator': ['tf'],
 'nx': ['300'],
 're': ['100000']}
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_ext_force_scheme_comparison\ke (re=100000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_ext_force_scheme_comparison\decay (re=100000) .png
Figure saved to d:\IIT Bombay - Miscellaneous\Winter Project\ddp-thesis\manuscript\code_figures\tgv_ext_force_scheme_comparison\Velocity field (re=100000) (Last common t_f).png
